In [ ]:
import matplotlib.pyplot as plt # for images and metrics visualization
import numpy as np # to manipulation nd arrays 
import tensorflow as tf # for tensors manipulation and deep learning models developing
import pathlib # for paths manipulation

from tensorflow import keras # keras is a high level api to develop DL architectures
from tensorflow.keras import layers # the modules to create different model layers
from tensorflow.keras.models import Sequential # to instansiate the model architecture

In [ ]:
# store directories paths 
train_dir = pathlib.Path("/kaggle/input/emotion-detection-fer/train")  
test_dir = pathlib.Path("/kaggle/input/emotion-detection-fer/test") 

# count the number of images in each folder (train / test)
train_image_count = len(list(train_dir.glob('*/*.png')))  
test_image_count = len(list(test_dir.glob('*/*.png')))
print(train_image_count)
print(test_image_count)

In [ ]:
# initiate our hyperparameters
batch_size = 32
img_height = 48
img_width = 48

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(train_dir,validation_split=0.2,subset="training",seed=123,
                                                       image_size=(img_height, img_width),batch_size=batch_size)
val_ds = tf.keras.utils.image_dataset_from_directory(train_dir,validation_split=0.2,subset="validation",seed=123,image_size=(img_height, img_width),batch_size=batch_size)
test_ds = tf.keras.utils.image_dataset_from_directory(test_dir, seed=123,image_size=(img_height, img_width),batch_size=batch_size)

In [ ]:
# Extract classes 
class_names = train_ds.class_names
print(class_names)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")

In [ ]:
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, name="outputs")
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
earlyStoppingCallback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                         patience=5,
                                                         verbose= 1 ,
                                                         restore_best_weights=True
                                                        )

epochs = 10
history = model.fit(train_ds,validation_data=val_ds,epochs=epochs, callbacks= [earlyStoppingCallback])


In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_ds)

# Print the loss and accuracy
print("Loss: ", loss)
print("Accuracy: ", accuracy)

In [ ]:
im_path = "/kaggle/input/emotion-detection-fer/test/happy/im5.png"
img = tf.keras.utils.load_img(
    im_path, target_size=(img_height, img_width)
)
print(img.size)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch
print(img_array.shape)
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.save("EmotionDetecttor_2")

In [ ]:
recons_model = tf.keras.models.load_model("EmotionDetecttor_2")

In [ ]:
pr = recons_model.predict(img_array)
sc=tf.nn.softmax(pr[0])
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(sc)], 100 * np.max(sc))
)

In [ ]:
from keras.applications.mobilenet import MobileNet
from keras.layers import Flatten,Activation, Dense

In [ ]:
base_mobilenet_model = MobileNet(weights = None,
                                 include_top = False,
                                 input_shape = (48, 48,3)
                                 )

#Build the Custom mobilenet Model

def mobilenet_custom():
    model = Sequential()
    #add mobilenet conv model
    model.add(base_mobilenet_model)
    #add new layers
    model.add(Flatten())
    model.add(Dense(7,  kernel_initializer='normal'))
    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])    
    return model

In [ ]:
model = mobilenet_custom()
model.summary()

In [ ]:
history = model.fit(train_ds,validation_data=val_ds,epochs=epochs, callbacks= [earlyStoppingCallback])

In [ ]:
im_path = "/kaggle/input/emotion-detection-fer/test/happy/im5.png"
img = tf.keras.utils.load_img(
    im_path, target_size=(img_height, img_width)
)
print(img.size)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch
print(img_array.shape)
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()